In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter_crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "Rt$&9012!@"
host = "localhost"
port = 27017
database = "aac"
collection = "animals"

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port, database, collection)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# html layout for the app
app.layout = html.Div([
    # app logo and name identifier
    html.Center(html.Img(src="data:image/png;base64,{}".format(encoded_image.decode("utf-8")),
                         style={"width": "300px"}), ),
    html.Center(html.B(html.H5('By Shayne Greene'))),
    html.Hr(),
    # Dropdown list filter for the rescue type selection
    html.Div(className='filter_container',
             style={'display': 'flex'},
             children=[
                 html.Div([
                     dcc.Dropdown(['Water', 'Mountain or Wilderness', 'Disaster or Individual Tracking'],
                                  id='filter-type',
                                  placeholder="Select a Rescue Type"),
                     html.Div(id='dd-output-container')
                 ], style={'width': '20%', 'margin-right': '100px auto', 'justify-content': 'start'}
                 ),
                 # Slider to select the amount of records to show at one time
                 html.Div([
                     dcc.Slider(10, 30, 5,
                                value=10,
                                marks={20: 'Records To Display'},
                                id='row-slider',
                                tooltip={"placement": "top", "always_visible": True}
                                ),
                     html.Div(id='slider-output-container')
                 ], style={'width': '20%'}
                 ),
             ]),

    html.Br(),

    html.Hr(),
    # The table that displays the data
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         row_selectable="single",
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10  # Defaulting to 10, but using a slider to adjust the amount you want to see
                         ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',

                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])


#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # Setting the query as empty in the case that the dropdown is reset / initial loading
    query = ""

    # When the Water type is selected
    if filter_type == 'Water':
        query = {
            "$and": [
                {"sex_upon_outcome": "Intact Female"},
                {
                    "$or": [
                        {"breed": "Labrador Retriever Mix"},
                        {"breed": "Chesapeake Bay Retriever"},
                        {"breed": "Newfoundland"}
                    ]
                },
                {
                    "age_upon_outcome_in_weeks": {
                        "$gte": 26,  # low range
                        "$lte": 156  # high range
                    }
                }
            ]
        }
    # filter for when Mountain or Wilderness is selected
    elif filter_type == "Mountain or Wilderness":
        query = {
            "$and": [
                {"sex_upon_outcome": "Intact Male"},
                {
                    "$or": [
                        {"breed": "German Shepherd"},
                        {"breed": "Alaskan Malamute"},
                        {"breed": "Old English Sheepdog"},
                        {"breed": "Siberian Husky"},
                        {"breed": "Rottweiler"}
                    ]
                },
                {
                    "age_upon_outcome_in_weeks": {
                        "$gte": 26,  # low range
                        "$lte": 156  # high range
                    }
                }
            ]
        }
    # filter for when Disaster or Individual Tracking is selected
    elif filter_type == "Disaster or Individual Tracking":
        query = {
            "$and": [
                {"sex_upon_outcome": "Intact Male"},
                {
                    "$or": [
                        {"breed": "German Shepherd"},
                        {"breed": "Doberman Pinscher"},
                        {"breed": "Golden Retriever"},
                        {"breed": "Bloodhound"},
                        {"breed": "Rottweiler"}
                    ]
                },
                {
                    "age_upon_outcome_in_weeks": {
                        "$gte": 20,  # low range
                        "$lte": 300  # high range
                    }
                }
            ]
        }

    # Setting the uery to display in the table
    df = pd.DataFrame.from_records(db.read(query))

    # Dropping the _id column if it exists in the df
    if "_id" in df.columns:
        df.drop(columns=['_id'], inplace=True)

    # Update the table with the new records
    return df.to_dict('records')


## Pie Chart ##
# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure=px.pie(dff, names='breed', title='Preferred Animals')
        )
    ]


# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This call back will adjust the amount of records that are shown in the table.
# min = 10, max = 30. Increments of 5
@app.callback(
    Output('datatable-id', 'page_size'),
    Input('row-slider', 'value'))
def update_table_page_size(slider_value):
    return slider_value


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None:
        return
    elif index is None:
        return

    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else:
        row = index[0]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server()